In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from numpy.f2py.auxfuncs import throw_error

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

import warnings



### Création de l'authentification API Spotify

In [11]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

SPOTIFY_CLIENT_ID = "ed9ece4a286d42f1aa834c82bed1f95e"
SPOTIFY_CLIENT_SECRET = "e1f250bb36914f3c9235c88df325eff6"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET,
    redirect_uri="http://localhost:8000/callback",
    scope="user-top-read playlist-read-private user-library-read playlist-modify-public playlist-modify-private"
))

In [10]:
import os

# Path to the cached token file
token_cache_path = ".cache"

# Check if the token cache file exists and delete it
if os.path.exists(token_cache_path):
    os.remove(token_cache_path)
    print("Spotify account disconnected.")
else:
    print("No Spotify account is currently connected.")

Spotify account disconnected.


### Get the user's top tracks

In [3]:
top_tracks = sp.current_user_top_tracks(limit=50, time_range='long_term')

print("Your top tracks:")

for idx, track in enumerate(top_tracks['items']):
    print(f"{idx + 1}: {track['name']} by {track['artists'][0]['name']}")
    

Your top tracks:
1: Where Is My Mind? - 2007 Remaster by Pixies
2: Jocelyn Flores by XXXTENTACION
3: Falling Down - Bonus Track by Lil Peep
4: Teardrop by Massive Attack
5: The Night We Met by Lord Huron
6: Évidemment by Lomepal
7: In This Dark Time by Aime Simone
8: death bed (coffee for your head) by Powfu
9: Paradise Circus by Massive Attack
10: Take Me To Church by Hozier
11: The Seed by AURORA
12: Reactor by Woodkid
13: Hasarder by Lomepal
14: Way down We Go by KALEO
15: Trop beau by Lomepal
16: Another Love by Tom Odell
17: San by Orelsan
18: Where's My Love by SYML
19: Rip X by iamjakehill
20: 7 Years by Lukas Graham
21: How to Save a Life by The Fray
22: Lucid Dreams by Juice WRLD
23: Sorry by Halsey
24: 505 by Arctic Monkeys
25: Sundance by Népal
26: Flash by Lomepal
27: Nique les clones, Pt. II by Nekfeu
28: Runaway by AURORA
29: Good News by Mac Miller
30: Jour meilleur by Orelsan
31: Si facile by Casseurs Flowters
32: Guns for Hire (from the series Arcane League of Legends)

### Get the track IDs to fetch audio features

In [4]:
track_ids = [track['id'] for track in top_tracks['items']]

print(f"Track IDs: {track_ids}")

Track IDs: ['7wCmS9TTVUcIhRalDYFgPy', '7m9OqQk4RVRkw9JJdeAw96', '4jvjzW7Hm0yK4LvvE0Paz9', '67Hna13dNDkZvBpTXRIaOJ', '3hRV0jL3vUpRrcy398teAU', '43yYuYeSOOupBHOkH9pRtF', '2tVJpUamLfNdUI7SiyPIFF', '7eJMfftS33KTjuF7lTsMCx', '2BndJYJQ17UcEeUFJP5JmY', '3dYD57lRAUcMHufyqn9GcI', '16tvIGzCNfRLVbm8G39DDo', '020srIDHnOnzyIhzGTcp7s', '6V40xLcDlYwZpz3oKIVNhL', '0y1QJc3SJVPKJ1OvFmFqe6', '4WB9OeauHXnGOMUK4s80gM', '7jtQIBanIiJOMS6RyCx6jZ', '36SHbxigbf4iSCHdQVgy6V', '1B62o4CbdL9ckGvwsz2cgn', '0j2BydmlwcMqffV08hFDyI', '0VQjTLPG8l3egUHmIxe265', '5fVZC9GiM4e8vu99W0Xf6J', '285pBltuF7vW8TeWk8hdRR', '44n97yHySt0Z9rqPaXgjCK', '0BxE4FqsDD1Ot4YuBXwAPp', '3CBjC9bcb5MzAnP4Hu9SVL', '3EjJfMN67Bcv3FCGR9UgBB', '67obelIiEq3vhpkolCCzZR', '1WKOnYtM6HtzYvEJlSEIv5', '1DWZUa5Mzf2BwzpHtgbHPY', '6VOUfCrjmxH62oijWgbC6O', '7GfmLz1shp0aMwkPYHan0y', '2H3HPGuPyhcbg5AqoQ4BWr', '1VGzxJnVQND7Cg5H5wGj14', '54fap9q6JOH7xNJBEfJch6', '62w3djsCFX85c5V6qfsI7Y', '4lriIG2vNqwDWzOj2I9rtj', '5tP7eNuBiAPCElikyAL8h0', '6cT2ovEBYIdVIdjRM532dp', 

### Load data_appended dataset and fetch the top tracks using ids

In [6]:
data = pd.read_csv("data/data_appended.csv")

top_tracks_data = data[data['id'].isin(track_ids)]

print("Top tracks data :")
print("Found a total of {} tracks".format(len(top_tracks_data)))
print(top_tracks_data)

track_to_analyse = top_tracks_data['id'].tolist()

print(track_to_analyse)
print("Tracks that are not in top track :")
for idx,track in enumerate(top_tracks['items']):
    if track['id'] not in top_tracks_data['id'].values:
        print(f"{idx}: {track['name']} by {track['artists'][0]['name']}")

Top tracks data :
Found a total of 23 tracks
        valence  year  acousticness                       artists  \
11217    0.6630  1978       0.03840                ['The Police']   
15298    0.2160  1998       0.01730            ['Massive Attack']   
15617    0.1650  2000       0.74800                  ['Coldplay']   
16609    0.3610  2005       0.26900                  ['The Fray']   
18901    0.3370  2016       0.57900                     ['KALEO']   
19006    0.4370  2017       0.46900              ['XXXTENTACION']   
19045    0.2220  2017       0.02750              ['Harry Styles']   
19187    0.3510  2017       0.91300                    ['Halsey']   
19207    0.2180  2018       0.34900                ['Juice WRLD']   
19223    0.2730  2018       0.01750  ['Lil Peep', 'XXXTENTACION']   
19245    0.2040  2018       0.61900                      ['Joji']   
19645    0.3480  2020       0.73100      ['Powfu', 'beabadoobee']   
35061    0.2950  2002       0.01450           ['Robbie Wil

In [40]:
#Throw an error to stop the execution here
throw_error()

TypeError: throw_error.__init__() missing 1 required positional argument: 'mess'

In [7]:
from useKeam import FindSimilarSong

f = FindSimilarSong()
result = f.find_songs_for_playlist(track_to_analyse)


### Create Spotify playlist

In [8]:
# Get user ID (required to create a playlist)
user_id = sp.current_user()['id']

# Create a new playlist
playlist_name = "AI-ify Playlist"
playlist_description = "This is a playlist created using AI-ify."
playlist = sp.user_playlist_create(user_id, playlist_name, public=True, description=playlist_description)

# Add tracks to the playlist
sp.playlist_add_items(playlist['id'], result)

{'snapshot_id': 'AAAAAn4FK2kBkcOWPwhJUXyp9rN9eLsg'}